<a href="https://colab.research.google.com/github/1804054Miraz/Paper-Publishing/blob/main/ML_apply_in_Frequency_Band_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
import numpy as np
import os
import pandas as pd
import statistics as st
import scipy.stats as scst
from scipy.stats import skew
from scipy.stats import kurtosis
from scipy.stats import entropy
from scipy.stats import normaltest
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import time
from sklearn.ensemble import BaggingClassifier

In [2]:
path = '/content/drive/MyDrive/Dataset/Features/Katz_Fd_Features_a_c.csv'
Katz_Fd_Features_a_c = pd.read_csv(path)
# Katz_Fd_Features_a_c = Katz_Fd_Features_a_c.replace(0, Katz_Fd_Features_a_c.mean(numeric_only=True))
Katz_Fd_Features_a_c = Katz_Fd_Features_a_c.drop(7620)
Katz_Fd_Features_a_c = Katz_Fd_Features_a_c.reset_index(drop=True)

In [3]:
Katz_Fd_Features_a_c

,FP2,label
0,1.655864,addicted
1,1.367933,addicted
2,1.584968,addicted
3,1.282758,addicted
4,1.461674,addicted
...,...,...
7616,1.819097,normal
7617,1.538744,normal
7618,1.716444,normal
7619,1.556286,normal


In [27]:
# Assuming 'new_dataframe' is your DataFrame
X = Katz_Fd_Features_a_c.iloc[:, :-1]
y = Katz_Fd_Features_a_c.iloc[:, -1]

# Encode the target variable if it's categorical
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Support Vector Machine (SVM)
# start_time = time.time()
svm_classifier = make_pipeline(StandardScaler(), SVC())
svm_predictions = cross_val_predict(svm_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
svm_accuracy = accuracy_score(y_encoded, svm_predictions)
# svm_precision = precision_score(y_encoded, svm_predictions)
# svm_recall = recall_score(y_encoded, svm_predictions)
# svm_f1 = f1_score(y_encoded, svm_predictions)
# svm_processing_time = time.time() - start_time

print("SVM Metrics:")
print("Accuracy:", svm_accuracy)
# print("Precision:", svm_precision)
# print("Recall:", svm_recall)
# print("F1 Score:", svm_f1)
# print("Processing Time:", svm_processing_time)

# XGBoost
# start_time = time.time()
xgb_classifier = XGBClassifier()
xgb_predictions = cross_val_predict(xgb_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
xgb_accuracy = accuracy_score(y_encoded, xgb_predictions)
# xgb_precision = precision_score(y_encoded, xgb_predictions)
# xgb_recall = recall_score(y_encoded, xgb_predictions)
# xgb_f1 = f1_score(y_encoded, xgb_predictions)
# xgb_processing_time = time.time() - start_time

print("\nXGBoost Metrics:")
print("Accuracy:", xgb_accuracy)
# print("Precision:", xgb_precision)
# print("Recall:", xgb_recall)
# print("F1 Score:", xgb_f1)
# print("Processing Time:", xgb_processing_time)

# Random Forest
# start_time = time.time()
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_predictions = cross_val_predict(rf_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
rf_accuracy = accuracy_score(y_encoded, rf_predictions)
# rf_precision = precision_score(y_encoded, rf_predictions)
# rf_recall = recall_score(y_encoded, rf_predictions)
# rf_f1 = f1_score(y_encoded, rf_predictions)
# rf_processing_time = time.time() - start_time

print("\nRandom Forest Metrics:")
print("Accuracy:", rf_accuracy)
# print("Precision:", rf_precision)
# print("Recall:", rf_recall)
# print("F1 Score:", rf_f1)
# print("Processing Time:", rf_processing_time)


# Ensemble Subspace of k-Nearest Neighbors
# start_time_ensemble_knn = time.time()
knn_classifier = KNeighborsClassifier(n_neighbors=5)
ensemble_knn_classifier = BaggingClassifier(base_estimator=knn_classifier, n_estimators=10, random_state=42)

ensemble_knn_scores = cross_val_predict(ensemble_knn_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
ensemble_knn_accuracy = accuracy_score(y_encoded, ensemble_knn_scores)
# ensemble_knn_precision = precision_score(y_encoded, ensemble_knn_scores)
# ensemble_knn_recall = recall_score(y_encoded, ensemble_knn_scores)
# ensemble_knn_f1 = f1_score(y_encoded, ensemble_knn_scores)
# ensemble_knn_processing_time = time.time() - start_time_ensemble_knn

print("\nEnsemble Subspace of k-Nearest Neighbors Metrics:")
print("Accuracy:", ensemble_knn_accuracy)
# print("Precision:", ensemble_knn_precision)
# print("Recall:", ensemble_knn_recall)
# print("F1 Score:", ensemble_knn_f1)
# print("Processing Time:", ensemble_knn_processing_time)

# LSTM Model
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, input_shape=input_shape, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Convert DataFrame to numpy array
X_np = X.values
X_np = X_np.reshape((X_np.shape[0], 1, X_np.shape[1]))

# 10-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

accuracy_scores = []
# precision_scores = []
# recall_scores = []
# f1_scores = []

for train_idx, test_idx in kfold.split(X_np):
    X_train, X_test = X_np[train_idx], X_np[test_idx]
    y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]

    # Create and fit LSTM model
    input_shape = (X_train.shape[1], X_train.shape[2])
    lstm_model = create_lstm_model(input_shape)
    lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

    # Predictions
    y_pred_probs = lstm_model.predict(X_test)
    y_pred = (y_pred_probs > 0.5).astype(int).flatten()

    # Evaluate metrics
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    # precision_scores.append(precision_score(y_test, y_pred))
    # recall_scores.append(recall_score(y_test, y_pred))
    # f1_scores.append(f1_score(y_test, y_pred))

print("\nLSTM Metrics:")
print("LSTM Model Accuracy:", np.mean(accuracy_scores))
# print("LSTM Model Precision:", np.mean(precision_scores))
# print("LSTM Model Recall:", np.mean(recall_scores))
# print("LSTM Model F1 Score:", np.mean(f1_scores))

SVM Metrics:
Accuracy: 0.578401784542711

XGBoost Metrics:
Accuracy: 0.5651489305865371

Random Forest Metrics:
Accuracy: 0.526702532476053


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/en


Ensemble Subspace of k-Nearest Neighbors Metrics:
Accuracy: 0.5441543104579452
24/24 [==============================] - 0s 3ms/step

LSTM Metrics:
LSTM Model Accuracy: 0.5643632848646213


In [28]:
import lightgbm as lgb
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming 'new_dataframe' is your DataFrame
X = Katz_Fd_Features_a_c.iloc[:, :-1]
y = Katz_Fd_Features_a_c.iloc[:, -1]

# Encode the target variable if it's categorical
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# LightGBM
lgb_classifier = lgb.LGBMClassifier()
lgb_predictions = cross_val_predict(lgb_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
lgb_accuracy = accuracy_score(y_encoded, lgb_predictions)
# lgb_precision = precision_score(y_encoded, lgb_predictions)
# lgb_recall = recall_score(y_encoded, lgb_predictions)
# lgb_f1 = f1_score(y_encoded, lgb_predictions)

print("\nLightGBM Metrics:")
print("Accuracy:", lgb_accuracy)
# print("Precision:", lgb_precision)
# print("Recall:", lgb_recall)
# print("F1 Score:", lgb_f1)

from catboost import CatBoostClassifier

# CatBoost
catboost_classifier = CatBoostClassifier(verbose=0)
catboost_predictions = cross_val_predict(catboost_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
catboost_accuracy = accuracy_score(y_encoded, catboost_predictions)
# catboost_precision = precision_score(y_encoded, catboost_predictions)
# catboost_recall = recall_score(y_encoded, catboost_predictions)
# catboost_f1 = f1_score(y_encoded, catboost_predictions)

print("\nCatBoost Metrics:")
print("Accuracy:", catboost_accuracy)
# print("Precision:", catboost_precision)
# print("Recall:", catboost_recall)
# print("F1 Score:", catboost_f1)


[LightGBM] [Info] Number of positive: 3438, number of negative: 3420
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000194 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 6858, number of used features: 1
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501312 -> initscore=0.005249
[LightGBM] [Info] Start training from score 0.005249
[LightGBM] [Info] Number of positive: 3445, number of negative: 3414
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000177 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 6859, number of used features: 1
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502260 -> initscore=0.009039
[LightGBM] [Info] Start training from score 0.009039
[LightGBM] [Info] Number o

In [29]:
import numpy as np
from sklearn.model_selection import cross_val_score, KFold, cross_validate
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

# Assuming 'new_dataframe' is your DataFrame
X = Katz_Fd_Features_a_c.iloc[:, :-1]
y = Katz_Fd_Features_a_c.iloc[:, -1]

# Encode the target variable if it's categorical
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Define individual models
xgb_classifier = XGBClassifier()
lgb_classifier = lgb.LGBMClassifier()
catboost_classifier = CatBoostClassifier(verbose=0)

# Define voting classifier
voting_classifier = VotingClassifier(
    estimators=[
        ('xgb', xgb_classifier),
        ('lgb', lgb_classifier),
        ('catboost', catboost_classifier)
    ],
    voting='soft'  # 'hard' for majority voting, 'soft' for weighted probabilities
)

# Define k-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Evaluate the model using cross-validation
scoring = {'accuracy': make_scorer(accuracy_score)}
          #  'precision': make_scorer(precision_score),
          #  'recall': make_scorer(recall_score),
          #  'f1': make_scorer(f1_score)


voting_scores = cross_validate(voting_classifier, X, y_encoded, cv=kf, scoring=scoring)

print("\nVoting Ensemble Metrics:")
print("Accuracy:", np.mean(voting_scores['test_accuracy']))
# print("Precision:", np.mean(voting_scores['test_precision']))
# print("Recall:", np.mean(voting_scores['test_recall']))
# print("F1 Score:", np.mean(voting_scores['test_f1']))


[LightGBM] [Info] Number of positive: 3438, number of negative: 3420
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000181 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 6858, number of used features: 1
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501312 -> initscore=0.005249
[LightGBM] [Info] Start training from score 0.005249
[LightGBM] [Info] Number of positive: 3445, number of negative: 3414
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000187 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 6859, number of used features: 1
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502260 -> initscore=0.009039
[LightGBM] [Info] Start training from score 0.009039
[LightGBM] [Info] Number o

In [4]:
path = '/content/drive/MyDrive/Dataset/Features/merged_alpha_a_c.csv'
merged_alpha_a_c = pd.read_csv(path)
# merged_alpha_a_c=merged_alpha_a_c.drop(columns=['X_PS','Y_PS','nd_PS'], axis=False)
# merged_alpha_a_c = merged_alpha_a_c.replace(0, merged_alpha_a_c.mean(numeric_only=True))
merged_alpha_a_c = merged_alpha_a_c.drop(7620)
merged_alpha_a_c = merged_alpha_a_c.reset_index(drop=True)

In [5]:
path = '/content/drive/MyDrive/Dataset/Features/merged_beta_a_c.csv'
merged_beta_a_c = pd.read_csv(path)
# merged_beta_a_c=merged_beta_a_c.drop(columns=['X_PS','Y_PS','nd_PS'], axis=False)
# merged_beta_a_c = merged_beta_a_c.replace(0, merged_beta_a_c.mean(numeric_only=True))
merged_beta_a_c = merged_beta_a_c.drop(7620)
merged_beta_a_c = merged_beta_a_c.reset_index(drop=True)

In [6]:
path = '/content/drive/MyDrive/Dataset/Features/merged_delta_a_c.csv'
merged_delta_a_c = pd.read_csv(path)
# merged_delta_a_c=merged_delta_a_c.drop(columns=['X_PS','Y_PS','nd_PS'], axis=False)
# merged_delta_a_c = merged_delta_a_c.replace(0, merged_delta_a_c.mean(numeric_only=True))
merged_delta_a_c = merged_delta_a_c.drop(7620)
merged_delta_a_c = merged_delta_a_c.reset_index(drop=True)

In [7]:
path = '/content/drive/MyDrive/Dataset/Features/merged_gamma_a_c.csv'
merged_gamma_a_c = pd.read_csv(path)
# merged_gamma_a_c=merged_gamma_a_c.drop(columns=['X_PS','Y_PS','nd_PS'], axis=False)
# merged_gamma_a_c = merged_gamma_a_c.replace(0, merged_gamma_a_c.mean(numeric_only=True))
merged_gamma_a_c = merged_gamma_a_c.drop(7620)
merged_gamma_a_c = merged_gamma_a_c.reset_index(drop=True)

In [8]:
path = '/content/drive/MyDrive/Dataset/Features/merged_theta_a_c.csv'
merged_theta_a_c = pd.read_csv(path)
# merged_theta_a_c=merged_theta_a_c.drop(columns=['X_PS','Y_PS','nd_PS'], axis=False)
# merged_theta_a_c = merged_theta_a_c.replace(0, merged_theta_a_c.mean(numeric_only=True))
merged_theta_a_c = merged_theta_a_c.drop(7620)
merged_theta_a_c = merged_theta_a_c.reset_index(drop=True)

In [9]:
merged_gamma_a_c

,FP1_PS,FP2_PS,F7_PS,F8_PS,AF1_PS,AF2_PS,FZ_PS,F4_PS,F3_PS,FC6_PS,...,PO8_PS,FCZ_PS,POZ_PS,OZ_PS,P2_PS,P1_PS,CPZ_PS,nd_PS,Y_PS,label
0,5.867765,16.136485,14.419578,107.004679,4.771000,10.412779,2.264702,4.967259,4.718617,2.227520,...,31.407003,0.639929,3.175360,7.257440,1.171091,1.727645,0.303052,5.626485,6.090333,addicted
1,4.788753,11.568614,14.381443,75.048537,2.176075,8.500067,1.558222,4.906503,3.750594,1.237990,...,31.837468,0.481474,1.360249,3.002968,0.452312,0.873387,0.169722,4.693551,3.472028,addicted
2,6.297802,29.544646,13.712807,97.931325,4.131878,13.930406,1.443750,4.954165,4.627834,2.631432,...,39.507001,0.403271,1.391440,2.908869,0.790318,1.077771,0.227142,6.266331,3.612442,addicted
3,4.348968,9.195677,10.247385,62.184384,2.506088,6.808748,1.321086,3.921911,3.681952,1.816489,...,30.478540,0.450695,1.121842,2.162650,0.753562,0.999099,0.180620,4.211706,6.950914,addicted
4,7.698942,15.086431,20.116347,61.128746,6.571280,8.982171,3.578223,4.040258,6.796313,2.204725,...,55.854653,1.011451,3.663678,5.646425,1.918202,2.340282,0.437771,7.413797,7.107321,addicted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7616,1.684572,1.623761,9.112097,2.782111,1.144432,0.884217,1.269359,0.880600,1.205278,0.957745,...,2.408939,0.174153,0.867184,1.234990,0.535141,0.633024,0.261792,1.058754,6.116856,normal
7617,2.437103,2.691105,9.169849,2.129555,1.467451,1.610917,0.581611,2.188392,2.580653,1.218395,...,5.024580,0.227139,3.721454,4.377724,1.713244,1.729774,0.453551,5.866098,3.061931,normal
7618,0.897495,1.245264,3.604244,3.566347,1.042233,1.365339,0.834950,1.492575,0.902565,1.093884,...,3.040292,0.398802,3.180998,2.880482,2.019003,1.853624,0.785453,2.408728,3.512361,normal
7619,1.251944,1.448347,6.375324,4.212090,0.632365,0.651843,0.297628,0.621490,0.765336,1.009508,...,2.070562,0.132006,1.298873,1.126842,0.855509,0.550932,0.247462,1.363607,3.663259,normal


In [ ]:
# merged_gamma_katz_a_c = merged_gamma_katz_a_c.drop(7620)
# If you want to reset the index after removing the row
# merged_gamma_katz_a_c = merged_gamma_katz_a_c.reset_index(drop=True)

In [ ]:
# merged_beta_gamma_a_c = pd.concat([merged_beta_a_c,merged_gamma_a_c], axis=1)
# # merged_beta_gamma_a_c

In [ ]:
# merged_beta_gamma_a_c.drop('label', axis=1, inplace=True)

In [ ]:
# merged_beta_gamma_a_c['label']='normal'
# merged_beta_gamma_a_c.loc[0:3810, 'label'] = 'addicted'


In [18]:
# Assuming 'new_dataframe' is your DataFrame
X = merged_alpha_a_c.iloc[:, :-1]
y = merged_alpha_a_c.iloc[:, -1]

# Encode the target variable if it's categorical
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Support Vector Machine (SVM)
# start_time = time.time()
svm_classifier = make_pipeline(StandardScaler(), SVC())
svm_predictions = cross_val_predict(svm_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
svm_accuracy = accuracy_score(y_encoded, svm_predictions)
# svm_precision = precision_score(y_encoded, svm_predictions)
# svm_recall = recall_score(y_encoded, svm_predictions)
# svm_f1 = f1_score(y_encoded, svm_predictions)
# svm_processing_time = time.time() - start_time

print("SVM Metrics:")
print("Accuracy:", svm_accuracy)
# print("Precision:", svm_precision)
# print("Recall:", svm_recall)
# print("F1 Score:", svm_f1)
# print("Processing Time:", svm_processing_time)

# XGBoost
# start_time = time.time()
xgb_classifier = XGBClassifier()
xgb_predictions = cross_val_predict(xgb_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
xgb_accuracy = accuracy_score(y_encoded, xgb_predictions)
# xgb_precision = precision_score(y_encoded, xgb_predictions)
# xgb_recall = recall_score(y_encoded, xgb_predictions)
# xgb_f1 = f1_score(y_encoded, xgb_predictions)
# xgb_processing_time = time.time() - start_time

print("\nXGBoost Metrics:")
print("Accuracy:", xgb_accuracy)
# print("Precision:", xgb_precision)
# print("Recall:", xgb_recall)
# print("F1 Score:", xgb_f1)
# print("Processing Time:", xgb_processing_time)

# Random Forest
# start_time = time.time()
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_predictions = cross_val_predict(rf_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
rf_accuracy = accuracy_score(y_encoded, rf_predictions)
# rf_precision = precision_score(y_encoded, rf_predictions)
# rf_recall = recall_score(y_encoded, rf_predictions)
# rf_f1 = f1_score(y_encoded, rf_predictions)
# rf_processing_time = time.time() - start_time

print("\nRandom Forest Metrics:")
print("Accuracy:", rf_accuracy)
# print("Precision:", rf_precision)
# print("Recall:", rf_recall)
# print("F1 Score:", rf_f1)
# print("Processing Time:", rf_processing_time)


# Ensemble Subspace of k-Nearest Neighbors
# start_time_ensemble_knn = time.time()
knn_classifier = KNeighborsClassifier(n_neighbors=5)
ensemble_knn_classifier = BaggingClassifier(base_estimator=knn_classifier, n_estimators=10, random_state=42)

ensemble_knn_scores = cross_val_predict(ensemble_knn_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
ensemble_knn_accuracy = accuracy_score(y_encoded, ensemble_knn_scores)
# ensemble_knn_precision = precision_score(y_encoded, ensemble_knn_scores)
# ensemble_knn_recall = recall_score(y_encoded, ensemble_knn_scores)
# ensemble_knn_f1 = f1_score(y_encoded, ensemble_knn_scores)
# ensemble_knn_processing_time = time.time() - start_time_ensemble_knn

print("\nEnsemble Subspace of k-Nearest Neighbors Metrics:")
print("Accuracy:", ensemble_knn_accuracy)
# print("Precision:", ensemble_knn_precision)
# print("Recall:", ensemble_knn_recall)
# print("F1 Score:", ensemble_knn_f1)
# print("Processing Time:", ensemble_knn_processing_time)

# LSTM Model
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, input_shape=input_shape, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Convert DataFrame to numpy array
X_np = X.values
X_np = X_np.reshape((X_np.shape[0], 1, X_np.shape[1]))

# 10-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

accuracy_scores = []
# precision_scores = []
# recall_scores = []
# f1_scores = []

for train_idx, test_idx in kfold.split(X_np):
    X_train, X_test = X_np[train_idx], X_np[test_idx]
    y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]

    # Create and fit LSTM model
    input_shape = (X_train.shape[1], X_train.shape[2])
    lstm_model = create_lstm_model(input_shape)
    lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

    # Predictions
    y_pred_probs = lstm_model.predict(X_test)
    y_pred = (y_pred_probs > 0.5).astype(int).flatten()

    # Evaluate metrics
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    # precision_scores.append(precision_score(y_test, y_pred))
    # recall_scores.append(recall_score(y_test, y_pred))
    # f1_scores.append(f1_score(y_test, y_pred))

print("\nLSTM Metrics:")
print("LSTM Model Accuracy:", np.mean(accuracy_scores))
# print("LSTM Model Precision:", np.mean(precision_scores))
# print("LSTM Model Recall:", np.mean(recall_scores))
# print("LSTM Model F1 Score:", np.mean(f1_scores))

SVM Metrics:
Accuracy: 0.7289069675895552

XGBoost Metrics:
Accuracy: 0.7857236583125574

Random Forest Metrics:
Accuracy: 0.7537068626164546


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/en


Ensemble Subspace of k-Nearest Neighbors Metrics:
Accuracy: 0.7299566985959848
24/24 [==============================] - 0s 2ms/step

LSTM Metrics:
LSTM Model Accuracy: 0.6499103896416618


In [19]:
import lightgbm as lgb
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming 'new_dataframe' is your DataFrame
X = merged_alpha_a_c.iloc[:, :-1]
y = merged_alpha_a_c.iloc[:, -1]

# Encode the target variable if it's categorical
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# LightGBM
lgb_classifier = lgb.LGBMClassifier()
lgb_predictions = cross_val_predict(lgb_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
lgb_accuracy = accuracy_score(y_encoded, lgb_predictions)
# lgb_precision = precision_score(y_encoded, lgb_predictions)
# lgb_recall = recall_score(y_encoded, lgb_predictions)
# lgb_f1 = f1_score(y_encoded, lgb_predictions)

print("\nLightGBM Metrics:")
print("Accuracy:", lgb_accuracy)
# print("Precision:", lgb_precision)
# print("Recall:", lgb_recall)
# print("F1 Score:", lgb_f1)

from catboost import CatBoostClassifier

# CatBoost
catboost_classifier = CatBoostClassifier(verbose=0)
catboost_predictions = cross_val_predict(catboost_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
catboost_accuracy = accuracy_score(y_encoded, catboost_predictions)
# catboost_precision = precision_score(y_encoded, catboost_predictions)
# catboost_recall = recall_score(y_encoded, catboost_predictions)
# catboost_f1 = f1_score(y_encoded, catboost_predictions)

print("\nCatBoost Metrics:")
print("Accuracy:", catboost_accuracy)
# print("Precision:", catboost_precision)
# print("Recall:", catboost_recall)
# print("F1 Score:", catboost_f1)


[LightGBM] [Info] Number of positive: 3438, number of negative: 3420
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008421 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16320
[LightGBM] [Info] Number of data points in the train set: 6858, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501312 -> initscore=0.005249
[LightGBM] [Info] Start training from score 0.005249
[LightGBM] [Info] Number of positive: 3445, number of negative: 3414
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005683 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16320
[LightGBM] [Info] Number of data points in the train set: 6859, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502260 -> initscore=0.009039
[LightGBM] [Info] Start training from score 0.009039
[LightGBM] [Info] Nu

In [20]:
import numpy as np
from sklearn.model_selection import cross_val_score, KFold, cross_validate
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

# Assuming 'new_dataframe' is your DataFrame
X = merged_alpha_a_c.iloc[:, :-1]
y = merged_alpha_a_c.iloc[:, -1]

# Encode the target variable if it's categorical
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Define individual models
xgb_classifier = XGBClassifier()
lgb_classifier = lgb.LGBMClassifier()
catboost_classifier = CatBoostClassifier(verbose=0)

# Define voting classifier
voting_classifier = VotingClassifier(
    estimators=[
        ('xgb', xgb_classifier),
        ('lgb', lgb_classifier),
        ('catboost', catboost_classifier)
    ],
    voting='soft'  # 'hard' for majority voting, 'soft' for weighted probabilities
)

# Define k-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Evaluate the model using cross-validation
scoring = {'accuracy': make_scorer(accuracy_score)}
          #  'precision': make_scorer(precision_score),
          #  'recall': make_scorer(recall_score),
          #  'f1': make_scorer(f1_score)


voting_scores = cross_validate(voting_classifier, X, y_encoded, cv=kf, scoring=scoring)

print("\nVoting Ensemble Metrics:")
print("Accuracy:", np.mean(voting_scores['test_accuracy']))
# print("Precision:", np.mean(voting_scores['test_precision']))
# print("Recall:", np.mean(voting_scores['test_recall']))
# print("F1 Score:", np.mean(voting_scores['test_f1']))


[LightGBM] [Info] Number of positive: 3438, number of negative: 3420
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005850 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16320
[LightGBM] [Info] Number of data points in the train set: 6858, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501312 -> initscore=0.005249
[LightGBM] [Info] Start training from score 0.005249
[LightGBM] [Info] Number of positive: 3445, number of negative: 3414
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005879 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16320
[LightGBM] [Info] Number of data points in the train set: 6859, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502260 -> initscore=0.009039
[LightGBM] [Info] Start training from score 0.009039
[LightGBM] [Info] Nu

In [21]:
# Assuming 'new_dataframe' is your DataFrame
X = merged_beta_a_c.iloc[:, :-1]
y = merged_beta_a_c.iloc[:, -1]

# Encode the target variable if it's categorical
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Support Vector Machine (SVM)
# start_time = time.time()
svm_classifier = make_pipeline(StandardScaler(), SVC())
svm_predictions = cross_val_predict(svm_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
svm_accuracy = accuracy_score(y_encoded, svm_predictions)
# svm_precision = precision_score(y_encoded, svm_predictions)
# svm_recall = recall_score(y_encoded, svm_predictions)
# svm_f1 = f1_score(y_encoded, svm_predictions)
# svm_processing_time = time.time() - start_time

print("SVM Metrics:")
print("Accuracy:", svm_accuracy)
# print("Precision:", svm_precision)
# print("Recall:", svm_recall)
# print("F1 Score:", svm_f1)
# print("Processing Time:", svm_processing_time)

# XGBoost
# start_time = time.time()
xgb_classifier = XGBClassifier()
xgb_predictions = cross_val_predict(xgb_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
xgb_accuracy = accuracy_score(y_encoded, xgb_predictions)
# xgb_precision = precision_score(y_encoded, xgb_predictions)
# xgb_recall = recall_score(y_encoded, xgb_predictions)
# xgb_f1 = f1_score(y_encoded, xgb_predictions)
# xgb_processing_time = time.time() - start_time

print("\nXGBoost Metrics:")
print("Accuracy:", xgb_accuracy)
# print("Precision:", xgb_precision)
# print("Recall:", xgb_recall)
# print("F1 Score:", xgb_f1)
# print("Processing Time:", xgb_processing_time)

# Random Forest
# start_time = time.time()
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_predictions = cross_val_predict(rf_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
rf_accuracy = accuracy_score(y_encoded, rf_predictions)
# rf_precision = precision_score(y_encoded, rf_predictions)
# rf_recall = recall_score(y_encoded, rf_predictions)
# rf_f1 = f1_score(y_encoded, rf_predictions)
# rf_processing_time = time.time() - start_time

print("\nRandom Forest Metrics:")
print("Accuracy:", rf_accuracy)
# print("Precision:", rf_precision)
# print("Recall:", rf_recall)
# print("F1 Score:", rf_f1)
# print("Processing Time:", rf_processing_time)


# Ensemble Subspace of k-Nearest Neighbors
# start_time_ensemble_knn = time.time()
knn_classifier = KNeighborsClassifier(n_neighbors=5)
ensemble_knn_classifier = BaggingClassifier(base_estimator=knn_classifier, n_estimators=10, random_state=42)

ensemble_knn_scores = cross_val_predict(ensemble_knn_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
ensemble_knn_accuracy = accuracy_score(y_encoded, ensemble_knn_scores)
# ensemble_knn_precision = precision_score(y_encoded, ensemble_knn_scores)
# ensemble_knn_recall = recall_score(y_encoded, ensemble_knn_scores)
# ensemble_knn_f1 = f1_score(y_encoded, ensemble_knn_scores)
# ensemble_knn_processing_time = time.time() - start_time_ensemble_knn

print("\nEnsemble Subspace of k-Nearest Neighbors Metrics:")
print("Accuracy:", ensemble_knn_accuracy)
# print("Precision:", ensemble_knn_precision)
# print("Recall:", ensemble_knn_recall)
# print("F1 Score:", ensemble_knn_f1)
# print("Processing Time:", ensemble_knn_processing_time)

# LSTM Model
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, input_shape=input_shape, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Convert DataFrame to numpy array
X_np = X.values
X_np = X_np.reshape((X_np.shape[0], 1, X_np.shape[1]))

# 10-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

accuracy_scores = []
# precision_scores = []
# recall_scores = []
# f1_scores = []

for train_idx, test_idx in kfold.split(X_np):
    X_train, X_test = X_np[train_idx], X_np[test_idx]
    y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]

    # Create and fit LSTM model
    input_shape = (X_train.shape[1], X_train.shape[2])
    lstm_model = create_lstm_model(input_shape)
    lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

    # Predictions
    y_pred_probs = lstm_model.predict(X_test)
    y_pred = (y_pred_probs > 0.5).astype(int).flatten()

    # Evaluate metrics
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    # precision_scores.append(precision_score(y_test, y_pred))
    # recall_scores.append(recall_score(y_test, y_pred))
    # f1_scores.append(f1_score(y_test, y_pred))

print("\nLSTM Metrics:")
print("LSTM Model Accuracy:", np.mean(accuracy_scores))
# print("LSTM Model Precision:", np.mean(precision_scores))
# print("LSTM Model Recall:", np.mean(recall_scores))
# print("LSTM Model F1 Score:", np.mean(f1_scores))

SVM Metrics:
Accuracy: 0.8375541267550191

XGBoost Metrics:
Accuracy: 0.9098543498228578

Random Forest Metrics:
Accuracy: 0.8632725364125443


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/en


Ensemble Subspace of k-Nearest Neighbors Metrics:
Accuracy: 0.8913528408345361
24/24 [==============================] - 0s 2ms/step

LSTM Metrics:
LSTM Model Accuracy: 0.8021241610853689


In [22]:
import lightgbm as lgb
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming 'new_dataframe' is your DataFrame
X = merged_beta_a_c.iloc[:, :-1]
y = merged_beta_a_c.iloc[:, -1]


# Encode the target variable if it's categorical
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# LightGBM
lgb_classifier = lgb.LGBMClassifier()
lgb_predictions = cross_val_predict(lgb_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
lgb_accuracy = accuracy_score(y_encoded, lgb_predictions)
# lgb_precision = precision_score(y_encoded, lgb_predictions)
# lgb_recall = recall_score(y_encoded, lgb_predictions)
# lgb_f1 = f1_score(y_encoded, lgb_predictions)

print("\nLightGBM Metrics:")
print("Accuracy:", lgb_accuracy)
# print("Precision:", lgb_precision)
# print("Recall:", lgb_recall)
# print("F1 Score:", lgb_f1)

from catboost import CatBoostClassifier

# CatBoost
catboost_classifier = CatBoostClassifier(verbose=0)
catboost_predictions = cross_val_predict(catboost_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
catboost_accuracy = accuracy_score(y_encoded, catboost_predictions)
# catboost_precision = precision_score(y_encoded, catboost_predictions)
# catboost_recall = recall_score(y_encoded, catboost_predictions)
# catboost_f1 = f1_score(y_encoded, catboost_predictions)

print("\nCatBoost Metrics:")
print("Accuracy:", catboost_accuracy)
# print("Precision:", catboost_precision)
# print("Recall:", catboost_recall)
# print("F1 Score:", catboost_f1)


[LightGBM] [Info] Number of positive: 3438, number of negative: 3420
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005583 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16320
[LightGBM] [Info] Number of data points in the train set: 6858, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501312 -> initscore=0.005249
[LightGBM] [Info] Start training from score 0.005249
[LightGBM] [Info] Number of positive: 3445, number of negative: 3414
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005759 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16320
[LightGBM] [Info] Number of data points in the train set: 6859, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502260 -> initscore=0.009039
[LightGBM] [Info] Start training from score 0.009039
[LightGBM] [Info] Nu

In [23]:
import numpy as np
from sklearn.model_selection import cross_val_score, KFold, cross_validate
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

# Assuming 'new_dataframe' is your DataFrame
X = merged_beta_a_c.iloc[:, :-1]
y = merged_beta_a_c.iloc[:, -1]


# Encode the target variable if it's categorical
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Define individual models
xgb_classifier = XGBClassifier()
lgb_classifier = lgb.LGBMClassifier()
catboost_classifier = CatBoostClassifier(verbose=0)

# Define voting classifier
voting_classifier = VotingClassifier(
    estimators=[
        ('xgb', xgb_classifier),
        ('lgb', lgb_classifier),
        ('catboost', catboost_classifier)
    ],
    voting='soft'  # 'hard' for majority voting, 'soft' for weighted probabilities
)

# Define k-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Evaluate the model using cross-validation
scoring = {'accuracy': make_scorer(accuracy_score)}
          #  'precision': make_scorer(precision_score),
          #  'recall': make_scorer(recall_score),
          #  'f1': make_scorer(f1_score)


voting_scores = cross_validate(voting_classifier, X, y_encoded, cv=kf, scoring=scoring)

print("\nVoting Ensemble Metrics:")
print("Accuracy:", np.mean(voting_scores['test_accuracy']))
# print("Precision:", np.mean(voting_scores['test_precision']))
# print("Recall:", np.mean(voting_scores['test_recall']))
# print("F1 Score:", np.mean(voting_scores['test_f1']))


[LightGBM] [Info] Number of positive: 3438, number of negative: 3420
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005796 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16320
[LightGBM] [Info] Number of data points in the train set: 6858, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501312 -> initscore=0.005249
[LightGBM] [Info] Start training from score 0.005249
[LightGBM] [Info] Number of positive: 3445, number of negative: 3414
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007779 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16320
[LightGBM] [Info] Number of data points in the train set: 6859, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502260 -> initscore=0.009039
[LightGBM] [Info] Start training from score 0.009039
[LightGBM] [Info] Nu

In [24]:
# Assuming 'new_dataframe' is your DataFrame
X = merged_delta_a_c.iloc[:, :-1]
y = merged_delta_a_c.iloc[:, -1]

# Encode the target variable if it's categorical
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Support Vector Machine (SVM)
# start_time = time.time()
svm_classifier = make_pipeline(StandardScaler(), SVC())
svm_predictions = cross_val_predict(svm_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
svm_accuracy = accuracy_score(y_encoded, svm_predictions)
# svm_precision = precision_score(y_encoded, svm_predictions)
# svm_recall = recall_score(y_encoded, svm_predictions)
# svm_f1 = f1_score(y_encoded, svm_predictions)
# svm_processing_time = time.time() - start_time

print("SVM Metrics:")
print("Accuracy:", svm_accuracy)
# print("Precision:", svm_precision)
# print("Recall:", svm_recall)
# print("F1 Score:", svm_f1)
# print("Processing Time:", svm_processing_time)

# XGBoost
# start_time = time.time()
xgb_classifier = XGBClassifier()
xgb_predictions = cross_val_predict(xgb_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
xgb_accuracy = accuracy_score(y_encoded, xgb_predictions)
# xgb_precision = precision_score(y_encoded, xgb_predictions)
# xgb_recall = recall_score(y_encoded, xgb_predictions)
# xgb_f1 = f1_score(y_encoded, xgb_predictions)
# xgb_processing_time = time.time() - start_time

print("\nXGBoost Metrics:")
print("Accuracy:", xgb_accuracy)
# print("Precision:", xgb_precision)
# print("Recall:", xgb_recall)
# print("F1 Score:", xgb_f1)
# print("Processing Time:", xgb_processing_time)

# Random Forest
# start_time = time.time()
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_predictions = cross_val_predict(rf_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
rf_accuracy = accuracy_score(y_encoded, rf_predictions)
# rf_precision = precision_score(y_encoded, rf_predictions)
# rf_recall = recall_score(y_encoded, rf_predictions)
# rf_f1 = f1_score(y_encoded, rf_predictions)
# rf_processing_time = time.time() - start_time

print("\nRandom Forest Metrics:")
print("Accuracy:", rf_accuracy)
# print("Precision:", rf_precision)
# print("Recall:", rf_recall)
# print("F1 Score:", rf_f1)
# print("Processing Time:", rf_processing_time)


# Ensemble Subspace of k-Nearest Neighbors
# start_time_ensemble_knn = time.time()
knn_classifier = KNeighborsClassifier(n_neighbors=5)
ensemble_knn_classifier = BaggingClassifier(base_estimator=knn_classifier, n_estimators=10, random_state=42)

ensemble_knn_scores = cross_val_predict(ensemble_knn_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
ensemble_knn_accuracy = accuracy_score(y_encoded, ensemble_knn_scores)
# ensemble_knn_precision = precision_score(y_encoded, ensemble_knn_scores)
# ensemble_knn_recall = recall_score(y_encoded, ensemble_knn_scores)
# ensemble_knn_f1 = f1_score(y_encoded, ensemble_knn_scores)
# ensemble_knn_processing_time = time.time() - start_time_ensemble_knn

print("\nEnsemble Subspace of k-Nearest Neighbors Metrics:")
print("Accuracy:", ensemble_knn_accuracy)
# print("Precision:", ensemble_knn_precision)
# print("Recall:", ensemble_knn_recall)
# print("F1 Score:", ensemble_knn_f1)
# print("Processing Time:", ensemble_knn_processing_time)

# LSTM Model
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, input_shape=input_shape, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Convert DataFrame to numpy array
X_np = X.values
X_np = X_np.reshape((X_np.shape[0], 1, X_np.shape[1]))

# 10-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

accuracy_scores = []
# precision_scores = []
# recall_scores = []
# f1_scores = []

for train_idx, test_idx in kfold.split(X_np):
    X_train, X_test = X_np[train_idx], X_np[test_idx]
    y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]

    # Create and fit LSTM model
    input_shape = (X_train.shape[1], X_train.shape[2])
    lstm_model = create_lstm_model(input_shape)
    lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

    # Predictions
    y_pred_probs = lstm_model.predict(X_test)
    y_pred = (y_pred_probs > 0.5).astype(int).flatten()

    # Evaluate metrics
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    # precision_scores.append(precision_score(y_test, y_pred))
    # recall_scores.append(recall_score(y_test, y_pred))
    # f1_scores.append(f1_score(y_test, y_pred))

print("\nLSTM Metrics:")
print("LSTM Model Accuracy:", np.mean(accuracy_scores))
# print("LSTM Model Precision:", np.mean(precision_scores))
# print("LSTM Model Recall:", np.mean(recall_scores))
# print("LSTM Model F1 Score:", np.mean(f1_scores))

SVM Metrics:
Accuracy: 0.6875738092113896

XGBoost Metrics:
Accuracy: 0.7479333420810917

Random Forest Metrics:
Accuracy: 0.7144731662511481


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/en


Ensemble Subspace of k-Nearest Neighbors Metrics:
Accuracy: 0.6538512006298386
24/24 [==============================] - 0s 4ms/step

LSTM Metrics:
LSTM Model Accuracy: 0.5365470944572295


In [25]:
import lightgbm as lgb
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming 'new_dataframe' is your DataFrame
X = merged_delta_a_c.iloc[:, :-1]
y = merged_delta_a_c.iloc[:, -1]

# Encode the target variable if it's categorical
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# LightGBM
lgb_classifier = lgb.LGBMClassifier()
lgb_predictions = cross_val_predict(lgb_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
lgb_accuracy = accuracy_score(y_encoded, lgb_predictions)
# lgb_precision = precision_score(y_encoded, lgb_predictions)
# lgb_recall = recall_score(y_encoded, lgb_predictions)
# lgb_f1 = f1_score(y_encoded, lgb_predictions)

print("\nLightGBM Metrics:")
print("Accuracy:", lgb_accuracy)
# print("Precision:", lgb_precision)
# print("Recall:", lgb_recall)
# print("F1 Score:", lgb_f1)

from catboost import CatBoostClassifier

# CatBoost
catboost_classifier = CatBoostClassifier(verbose=0)
catboost_predictions = cross_val_predict(catboost_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
catboost_accuracy = accuracy_score(y_encoded, catboost_predictions)
# catboost_precision = precision_score(y_encoded, catboost_predictions)
# catboost_recall = recall_score(y_encoded, catboost_predictions)
# catboost_f1 = f1_score(y_encoded, catboost_predictions)

print("\nCatBoost Metrics:")
print("Accuracy:", catboost_accuracy)
# print("Precision:", catboost_precision)
# print("Recall:", catboost_recall)
# print("F1 Score:", catboost_f1)


[LightGBM] [Info] Number of positive: 3438, number of negative: 3420
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009491 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16320
[LightGBM] [Info] Number of data points in the train set: 6858, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501312 -> initscore=0.005249
[LightGBM] [Info] Start training from score 0.005249
[LightGBM] [Info] Number of positive: 3445, number of negative: 3414
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010543 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16320
[LightGBM] [Info] Number of data points in the train set: 6859, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502260 -> initscore=0.009039
[LightGBM] [Info] Start training from score 0.009039
[LightGBM] [Info] Nu

In [26]:
import numpy as np
from sklearn.model_selection import cross_val_score, KFold, cross_validate
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

# Assuming 'new_dataframe' is your DataFrame
X = merged_delta_a_c.iloc[:, :-1]
y = merged_delta_a_c.iloc[:, -1]

# Encode the target variable if it's categorical
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Define individual models
xgb_classifier = XGBClassifier()
lgb_classifier = lgb.LGBMClassifier()
catboost_classifier = CatBoostClassifier(verbose=0)

# Define voting classifier
voting_classifier = VotingClassifier(
    estimators=[
        ('xgb', xgb_classifier),
        ('lgb', lgb_classifier),
        ('catboost', catboost_classifier)
    ],
    voting='soft'  # 'hard' for majority voting, 'soft' for weighted probabilities
)

# Define k-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Evaluate the model using cross-validation
scoring = {'accuracy': make_scorer(accuracy_score)}
          #  'precision': make_scorer(precision_score),
          #  'recall': make_scorer(recall_score),
          #  'f1': make_scorer(f1_score)


voting_scores = cross_validate(voting_classifier, X, y_encoded, cv=kf, scoring=scoring)

print("\nVoting Ensemble Metrics:")
print("Accuracy:", np.mean(voting_scores['test_accuracy']))
# print("Precision:", np.mean(voting_scores['test_precision']))
# print("Recall:", np.mean(voting_scores['test_recall']))
# print("F1 Score:", np.mean(voting_scores['test_f1']))


[LightGBM] [Info] Number of positive: 3438, number of negative: 3420
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005805 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16320
[LightGBM] [Info] Number of data points in the train set: 6858, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501312 -> initscore=0.005249
[LightGBM] [Info] Start training from score 0.005249
[LightGBM] [Info] Number of positive: 3445, number of negative: 3414
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014981 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16320
[LightGBM] [Info] Number of data points in the train set: 6859, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502260 -> initscore=0.009039
[LightGBM] [Info] Start training from score 0.009039
[LightGBM] [Info] Nu

In [11]:
# Assuming 'new_dataframe' is your DataFrame
X = merged_gamma_a_c.iloc[:, :-1]
y = merged_gamma_a_c.iloc[:, -1]

# Encode the target variable if it's categorical
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Support Vector Machine (SVM)
# start_time = time.time()
svm_classifier = make_pipeline(StandardScaler(), SVC())
svm_predictions = cross_val_predict(svm_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
svm_accuracy = accuracy_score(y_encoded, svm_predictions)
# svm_precision = precision_score(y_encoded, svm_predictions)
# svm_recall = recall_score(y_encoded, svm_predictions)
# svm_f1 = f1_score(y_encoded, svm_predictions)
# svm_processing_time = time.time() - start_time

print("SVM Metrics:")
print("Accuracy:", svm_accuracy)
# print("Precision:", svm_precision)
# print("Recall:", svm_recall)
# print("F1 Score:", svm_f1)
# print("Processing Time:", svm_processing_time)

# XGBoost
# start_time = time.time()
xgb_classifier = XGBClassifier()
xgb_predictions = cross_val_predict(xgb_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
xgb_accuracy = accuracy_score(y_encoded, xgb_predictions)
# xgb_precision = precision_score(y_encoded, xgb_predictions)
# xgb_recall = recall_score(y_encoded, xgb_predictions)
# xgb_f1 = f1_score(y_encoded, xgb_predictions)
# xgb_processing_time = time.time() - start_time

print("\nXGBoost Metrics:")
print("Accuracy:", xgb_accuracy)
# print("Precision:", xgb_precision)
# print("Recall:", xgb_recall)
# print("F1 Score:", xgb_f1)
# print("Processing Time:", xgb_processing_time)

# Random Forest
# start_time = time.time()
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_predictions = cross_val_predict(rf_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
rf_accuracy = accuracy_score(y_encoded, rf_predictions)
# rf_precision = precision_score(y_encoded, rf_predictions)
# rf_recall = recall_score(y_encoded, rf_predictions)
# rf_f1 = f1_score(y_encoded, rf_predictions)
# rf_processing_time = time.time() - start_time

print("\nRandom Forest Metrics:")
print("Accuracy:", rf_accuracy)
# print("Precision:", rf_precision)
# print("Recall:", rf_recall)
# print("F1 Score:", rf_f1)
# print("Processing Time:", rf_processing_time)


# Ensemble Subspace of k-Nearest Neighbors
# start_time_ensemble_knn = time.time()
knn_classifier = KNeighborsClassifier(n_neighbors=5)
ensemble_knn_classifier = BaggingClassifier(base_estimator=knn_classifier, n_estimators=10, random_state=42)

ensemble_knn_scores = cross_val_predict(ensemble_knn_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
ensemble_knn_accuracy = accuracy_score(y_encoded, ensemble_knn_scores)
# ensemble_knn_precision = precision_score(y_encoded, ensemble_knn_scores)
# ensemble_knn_recall = recall_score(y_encoded, ensemble_knn_scores)
# ensemble_knn_f1 = f1_score(y_encoded, ensemble_knn_scores)
# ensemble_knn_processing_time = time.time() - start_time_ensemble_knn

print("\nEnsemble Subspace of k-Nearest Neighbors Metrics:")
print("Accuracy:", ensemble_knn_accuracy)
# print("Precision:", ensemble_knn_precision)
# print("Recall:", ensemble_knn_recall)
# print("F1 Score:", ensemble_knn_f1)
# print("Processing Time:", ensemble_knn_processing_time)

# LSTM Model
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, input_shape=input_shape, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Convert DataFrame to numpy array
X_np = X.values
X_np = X_np.reshape((X_np.shape[0], 1, X_np.shape[1]))

# 10-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

accuracy_scores = []
# precision_scores = []
# recall_scores = []
# f1_scores = []

for train_idx, test_idx in kfold.split(X_np):
    X_train, X_test = X_np[train_idx], X_np[test_idx]
    y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]

    # Create and fit LSTM model
    input_shape = (X_train.shape[1], X_train.shape[2])
    lstm_model = create_lstm_model(input_shape)
    lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

    # Predictions
    y_pred_probs = lstm_model.predict(X_test)
    y_pred = (y_pred_probs > 0.5).astype(int).flatten()

    # Evaluate metrics
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    # precision_scores.append(precision_score(y_test, y_pred))
    # recall_scores.append(recall_score(y_test, y_pred))
    # f1_scores.append(f1_score(y_test, y_pred))

print("\nLSTM Metrics:")
print("LSTM Model Accuracy:", np.mean(accuracy_scores))
# print("LSTM Model Precision:", np.mean(precision_scores))
# print("LSTM Model Recall:", np.mean(recall_scores))
# print("LSTM Model F1 Score:", np.mean(f1_scores))

SVM Metrics:
Accuracy: 0.8463456239338669

XGBoost Metrics:
Accuracy: 0.9574858942396011

Random Forest Metrics:
Accuracy: 0.9278309933079648


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/en


Ensemble Subspace of k-Nearest Neighbors Metrics:
Accuracy: 0.9430520929011941
24/24 [==============================] - 1s 6ms/step

LSTM Metrics:
LSTM Model Accuracy: 0.9190390192051682


In [12]:
!pip install xgboost lightgbm catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 2.4 MB/s eta 0:00:00


In [13]:
import lightgbm as lgb
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming 'new_dataframe' is your DataFrame
X = merged_gamma_a_c.iloc[:, :-1]
y = merged_gamma_a_c.iloc[:, -1]

# Encode the target variable if it's categorical
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# LightGBM
lgb_classifier = lgb.LGBMClassifier()
lgb_predictions = cross_val_predict(lgb_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
lgb_accuracy = accuracy_score(y_encoded, lgb_predictions)
# lgb_precision = precision_score(y_encoded, lgb_predictions)
# lgb_recall = recall_score(y_encoded, lgb_predictions)
# lgb_f1 = f1_score(y_encoded, lgb_predictions)

print("\nLightGBM Metrics:")
print("Accuracy:", lgb_accuracy)
# print("Precision:", lgb_precision)
# print("Recall:", lgb_recall)
# print("F1 Score:", lgb_f1)

from catboost import CatBoostClassifier

# CatBoost
catboost_classifier = CatBoostClassifier(verbose=0)
catboost_predictions = cross_val_predict(catboost_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
catboost_accuracy = accuracy_score(y_encoded, catboost_predictions)
# catboost_precision = precision_score(y_encoded, catboost_predictions)
# catboost_recall = recall_score(y_encoded, catboost_predictions)
# catboost_f1 = f1_score(y_encoded, catboost_predictions)

print("\nCatBoost Metrics:")
print("Accuracy:", catboost_accuracy)
# print("Precision:", catboost_precision)
# print("Recall:", catboost_recall)
# print("F1 Score:", catboost_f1)


[LightGBM] [Info] Number of positive: 3438, number of negative: 3420
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010187 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16320
[LightGBM] [Info] Number of data points in the train set: 6858, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501312 -> initscore=0.005249
[LightGBM] [Info] Start training from score 0.005249
[LightGBM] [Info] Number of positive: 3445, number of negative: 3414
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005845 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16320
[LightGBM] [Info] Number of data points in the train set: 6859, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502260 -> initscore=0.009039
[LightGBM] [Info] Start training from score 0.009039
[LightGBM] [Info] Nu

In [14]:
import numpy as np
from sklearn.model_selection import cross_val_score, KFold, cross_validate
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

# Assuming 'new_dataframe' is your DataFrame
X = merged_gamma_a_c.iloc[:, :-1]
y = merged_gamma_a_c.iloc[:, -1]

# Encode the target variable if it's categorical
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Define individual models
xgb_classifier = XGBClassifier()
lgb_classifier = lgb.LGBMClassifier()
catboost_classifier = CatBoostClassifier(verbose=0)

# Define voting classifier
voting_classifier = VotingClassifier(
    estimators=[
        ('xgb', xgb_classifier),
        ('lgb', lgb_classifier),
        ('catboost', catboost_classifier)
    ],
    voting='soft'  # 'hard' for majority voting, 'soft' for weighted probabilities
)

# Define k-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Evaluate the model using cross-validation
scoring = {'accuracy': make_scorer(accuracy_score)}
          #  'precision': make_scorer(precision_score),
          #  'recall': make_scorer(recall_score),
          #  'f1': make_scorer(f1_score)


voting_scores = cross_validate(voting_classifier, X, y_encoded, cv=kf, scoring=scoring)

print("\nVoting Ensemble Metrics:")
print("Accuracy:", np.mean(voting_scores['test_accuracy']))
# print("Precision:", np.mean(voting_scores['test_precision']))
# print("Recall:", np.mean(voting_scores['test_recall']))
# print("F1 Score:", np.mean(voting_scores['test_f1']))


[LightGBM] [Info] Number of positive: 3438, number of negative: 3420
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009413 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16320
[LightGBM] [Info] Number of data points in the train set: 6858, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501312 -> initscore=0.005249
[LightGBM] [Info] Start training from score 0.005249
[LightGBM] [Info] Number of positive: 3445, number of negative: 3414
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005811 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16320
[LightGBM] [Info] Number of data points in the train set: 6859, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502260 -> initscore=0.009039
[LightGBM] [Info] Start training from score 0.009039
[LightGBM] [Info] Nu

In [15]:
# Assuming 'new_dataframe' is your DataFrame
X = merged_theta_a_c.iloc[:, :-1]
y = merged_theta_a_c.iloc[:, -1]

# Encode the target variable if it's categorical
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Support Vector Machine (SVM)
# start_time = time.time()
svm_classifier = make_pipeline(StandardScaler(), SVC())
svm_predictions = cross_val_predict(svm_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
svm_accuracy = accuracy_score(y_encoded, svm_predictions)
# svm_precision = precision_score(y_encoded, svm_predictions)
# svm_recall = recall_score(y_encoded, svm_predictions)
# svm_f1 = f1_score(y_encoded, svm_predictions)
# svm_processing_time = time.time() - start_time

print("SVM Metrics:")
print("Accuracy:", svm_accuracy)
# print("Precision:", svm_precision)
# print("Recall:", svm_recall)
# print("F1 Score:", svm_f1)
# print("Processing Time:", svm_processing_time)

# XGBoost
# start_time = time.time()
xgb_classifier = XGBClassifier()
xgb_predictions = cross_val_predict(xgb_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
xgb_accuracy = accuracy_score(y_encoded, xgb_predictions)
# xgb_precision = precision_score(y_encoded, xgb_predictions)
# xgb_recall = recall_score(y_encoded, xgb_predictions)
# xgb_f1 = f1_score(y_encoded, xgb_predictions)
# xgb_processing_time = time.time() - start_time

print("\nXGBoost Metrics:")
print("Accuracy:", xgb_accuracy)
# print("Precision:", xgb_precision)
# print("Recall:", xgb_recall)
# print("F1 Score:", xgb_f1)
# print("Processing Time:", xgb_processing_time)

# Random Forest
# start_time = time.time()
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_predictions = cross_val_predict(rf_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
rf_accuracy = accuracy_score(y_encoded, rf_predictions)
# rf_precision = precision_score(y_encoded, rf_predictions)
# rf_recall = recall_score(y_encoded, rf_predictions)
# rf_f1 = f1_score(y_encoded, rf_predictions)
# rf_processing_time = time.time() - start_time

print("\nRandom Forest Metrics:")
print("Accuracy:", rf_accuracy)
# print("Precision:", rf_precision)
# print("Recall:", rf_recall)
# print("F1 Score:", rf_f1)
# print("Processing Time:", rf_processing_time)


# Ensemble Subspace of k-Nearest Neighbors
# start_time_ensemble_knn = time.time()
knn_classifier = KNeighborsClassifier(n_neighbors=5)
ensemble_knn_classifier = BaggingClassifier(base_estimator=knn_classifier, n_estimators=10, random_state=42)

ensemble_knn_scores = cross_val_predict(ensemble_knn_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
ensemble_knn_accuracy = accuracy_score(y_encoded, ensemble_knn_scores)
# ensemble_knn_precision = precision_score(y_encoded, ensemble_knn_scores)
# ensemble_knn_recall = recall_score(y_encoded, ensemble_knn_scores)
# ensemble_knn_f1 = f1_score(y_encoded, ensemble_knn_scores)
# ensemble_knn_processing_time = time.time() - start_time_ensemble_knn

print("\nEnsemble Subspace of k-Nearest Neighbors Metrics:")
print("Accuracy:", ensemble_knn_accuracy)
# print("Precision:", ensemble_knn_precision)
# print("Recall:", ensemble_knn_recall)
# print("F1 Score:", ensemble_knn_f1)
# print("Processing Time:", ensemble_knn_processing_time)

# LSTM Model
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, input_shape=input_shape, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Convert DataFrame to numpy array
X_np = X.values
X_np = X_np.reshape((X_np.shape[0], 1, X_np.shape[1]))

# 10-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

accuracy_scores = []
# precision_scores = []
# recall_scores = []
# f1_scores = []

for train_idx, test_idx in kfold.split(X_np):
    X_train, X_test = X_np[train_idx], X_np[test_idx]
    y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]

    # Create and fit LSTM model
    input_shape = (X_train.shape[1], X_train.shape[2])
    lstm_model = create_lstm_model(input_shape)
    lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

    # Predictions
    y_pred_probs = lstm_model.predict(X_test)
    y_pred = (y_pred_probs > 0.5).astype(int).flatten()

    # Evaluate metrics
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    # precision_scores.append(precision_score(y_test, y_pred))
    # recall_scores.append(recall_score(y_test, y_pred))
    # f1_scores.append(f1_score(y_test, y_pred))

print("\nLSTM Metrics:")
print("LSTM Model Accuracy:", np.mean(accuracy_scores))
# print("LSTM Model Precision:", np.mean(precision_scores))
# print("LSTM Model Recall:", np.mean(recall_scores))
# print("LSTM Model F1 Score:", np.mean(f1_scores))

SVM Metrics:
Accuracy: 0.7380921138958142

XGBoost Metrics:
Accuracy: 0.7803437869046057

Random Forest Metrics:
Accuracy: 0.7500328040939509


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/en


Ensemble Subspace of k-Nearest Neighbors Metrics:
Accuracy: 0.7228710143025849
24/24 [==============================] - 0s 2ms/step

LSTM Metrics:
LSTM Model Accuracy: 0.5785413291228504


In [16]:
import lightgbm as lgb
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming 'new_dataframe' is your DataFrame
X = merged_theta_a_c.iloc[:, :-1]
y = merged_theta_a_c.iloc[:, -1]

# Encode the target variable if it's categorical
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# LightGBM
lgb_classifier = lgb.LGBMClassifier()
lgb_predictions = cross_val_predict(lgb_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
lgb_accuracy = accuracy_score(y_encoded, lgb_predictions)
# lgb_precision = precision_score(y_encoded, lgb_predictions)
# lgb_recall = recall_score(y_encoded, lgb_predictions)
# lgb_f1 = f1_score(y_encoded, lgb_predictions)

print("\nLightGBM Metrics:")
print("Accuracy:", lgb_accuracy)
# print("Precision:", lgb_precision)
# print("Recall:", lgb_recall)
# print("F1 Score:", lgb_f1)

from catboost import CatBoostClassifier

# CatBoost
catboost_classifier = CatBoostClassifier(verbose=0)
catboost_predictions = cross_val_predict(catboost_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
catboost_accuracy = accuracy_score(y_encoded, catboost_predictions)
# catboost_precision = precision_score(y_encoded, catboost_predictions)
# catboost_recall = recall_score(y_encoded, catboost_predictions)
# catboost_f1 = f1_score(y_encoded, catboost_predictions)

print("\nCatBoost Metrics:")
print("Accuracy:", catboost_accuracy)
# print("Precision:", catboost_precision)
# print("Recall:", catboost_recall)
# print("F1 Score:", catboost_f1)


[LightGBM] [Info] Number of positive: 3438, number of negative: 3420
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011604 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16320
[LightGBM] [Info] Number of data points in the train set: 6858, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501312 -> initscore=0.005249
[LightGBM] [Info] Start training from score 0.005249
[LightGBM] [Info] Number of positive: 3445, number of negative: 3414
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009102 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16320
[LightGBM] [Info] Number of data points in the train set: 6859, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502260 -> initscore=0.009039
[LightGBM] [Info] Start training from score 0.009039
[LightGBM] [Info] Nu

In [17]:
import numpy as np
from sklearn.model_selection import cross_val_score, KFold, cross_validate
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

# Assuming 'new_dataframe' is your DataFrame
X = merged_theta_a_c.iloc[:, :-1]
y = merged_theta_a_c.iloc[:, -1]

# Encode the target variable if it's categorical
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Define individual models
xgb_classifier = XGBClassifier()
lgb_classifier = lgb.LGBMClassifier()
catboost_classifier = CatBoostClassifier(verbose=0)

# Define voting classifier
voting_classifier = VotingClassifier(
    estimators=[
        ('xgb', xgb_classifier),
        ('lgb', lgb_classifier),
        ('catboost', catboost_classifier)
    ],
    voting='soft'  # 'hard' for majority voting, 'soft' for weighted probabilities
)

# Define k-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Evaluate the model using cross-validation
scoring = {'accuracy': make_scorer(accuracy_score)}
          #  'precision': make_scorer(precision_score),
          #  'recall': make_scorer(recall_score),
          #  'f1': make_scorer(f1_score)


voting_scores = cross_validate(voting_classifier, X, y_encoded, cv=kf, scoring=scoring)

print("\nVoting Ensemble Metrics:")
print("Accuracy:", np.mean(voting_scores['test_accuracy']))
# print("Precision:", np.mean(voting_scores['test_precision']))
# print("Recall:", np.mean(voting_scores['test_recall']))
# print("F1 Score:", np.mean(voting_scores['test_f1']))


[LightGBM] [Info] Number of positive: 3438, number of negative: 3420
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005628 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16320
[LightGBM] [Info] Number of data points in the train set: 6858, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501312 -> initscore=0.005249
[LightGBM] [Info] Start training from score 0.005249
[LightGBM] [Info] Number of positive: 3445, number of negative: 3414
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009545 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16320
[LightGBM] [Info] Number of data points in the train set: 6859, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502260 -> initscore=0.009039
[LightGBM] [Info] Start training from score 0.009039
[LightGBM] [Info] Nu

In [ ]:
merged_alpha_Katz_a_c = pd.concat([merged_alpha_a_c,Katz_Fd_Features_a_c], axis=1)
merged_alpha_Katz_a_c.drop('label', axis=1, inplace=True)
merged_alpha_Katz_a_c['label']='normal'
merged_alpha_Katz_a_c.loc[0:3810, 'label'] = 'addicted'
merged_alpha_Katz_a_c

merged_beta_Katz_a_c = pd.concat([merged_beta_a_c,Katz_Fd_Features_a_c], axis=1)
merged_beta_Katz_a_c.drop('label', axis=1, inplace=True)
merged_beta_Katz_a_c['label']='normal'
merged_beta_Katz_a_c.loc[0:3810, 'label'] = 'addicted'
# merged_beta_Katz_a_c

merged_gamma_Katz_a_c = pd.concat([merged_gamma_a_c,Katz_Fd_Features_a_c], axis=1)
merged_gamma_Katz_a_c.drop('label', axis=1, inplace=True)
merged_gamma_Katz_a_c['label']='normal'
merged_gamma_Katz_a_c.loc[0:3810, 'label'] = 'addicted'

merged_delta_Katz_a_c = pd.concat([merged_delta_a_c,Katz_Fd_Features_a_c], axis=1)
merged_delta_Katz_a_c.drop('label', axis=1, inplace=True)
merged_delta_Katz_a_c['label']='normal'
merged_delta_Katz_a_c.loc[0:3810, 'label'] = 'addicted'

merged_theta_Katz_a_c = pd.concat([merged_theta_a_c,Katz_Fd_Features_a_c], axis=1)
merged_theta_Katz_a_c.drop('label', axis=1, inplace=True)
merged_theta_Katz_a_c['label']='normal'
merged_theta_Katz_a_c.loc[0:3810, 'label'] = 'addicted'

